In [15]:
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model

import tensorflow as tf

In [16]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [17]:
import threading

In [18]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [19]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [20]:
import datetime
import os

In [21]:
import pandas as pd
import h5py

In [22]:
# Setup data seed
np.random.seed(3254)

In [23]:
# https://github.com/ternaus/kaggle_dstl_submission/

smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
#    return -K.log(jaccard_coef(y_true, y_pred))

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [24]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = BatchNormalization(axis=1)(c4)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    c4 = BatchNormalization(axis=1)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (p4)
    c5 = BatchNormalization(axis=1)(c5)
    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (c5)
    c5 = BatchNormalization(axis=1)(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = BatchNormalization(axis=1)(c6)
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)
    c6 = BatchNormalization(axis=1)(c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = BatchNormalization(axis=1)(c7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)
    c7 = BatchNormalization(axis=1)(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = BatchNormalization(axis=1)(c8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)
    c8 = BatchNormalization(axis=1)(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = BatchNormalization(axis=1)(c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization(axis=1)(c9)

    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [25]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [26]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/rgbtest/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                       repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw[:,:,:,0:3] / 65535
    
    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3092
{3092: 1.0}
composition with label != 0
Y_train 0.5410628019323671 0.1474718196457327
composition with label != 0
Y_val 0.6 0.19891964285714286
composition with label != 0
Y_test 0.5202312138728323 0.12354407514450867
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 3) (70, 112, 112, 3) (173, 112, 112, 3) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., steps_per_epoch=100, verbose=1, epochs=50, callbacks=[<keras.ca..., validation_data=(array([[[...)`


Epoch 1/50
100/100 [==============================] - 17s 166ms/step - loss: 5.3237 - binary_crossentropy: 0.4812 - jaccard_coef_int: 0.1190 - binary_accuracy: 0.8333 - val_loss: 7.5975 - val_binary_crossentropy: 0.8584 - val_jaccard_coef_int: 8.9991e-16 - val_binary_accuracy: 0.8011
Epoch 2/50
100/100 [==============================] - 9s 89ms/step - loss: 4.2376 - binary_crossentropy: 0.6064 - jaccard_coef_int: 0.2244 - binary_accuracy: 0.8414 - val_loss: 7.5676 - val_binary_crossentropy: 1.0527 - val_jaccard_coef_int: 0.0084 - val_binary_accuracy: 0.7932
Epoch 3/50
100/100 [==============================] - 9s 89ms/step - loss: 4.0300 - binary_crossentropy: 0.6783 - jaccard_coef_int: 0.2324 - binary_accuracy: 0.8474 - val_loss: 8.0266 - val_binary_crossentropy: 1.1170 - val_jaccard_coef_int: 0.0063 - val_binary_accuracy: 0.7926
Epoch 4/50
100/100 [==============================] - 9s 88ms/step - loss: 3.7217 - binary_crossentropy: 0.6999 - jaccard_coef_int: 0.2728 - binary_accuracy:

Epoch 7/50
100/100 [==============================] - 10s 104ms/step - loss: 3.4120 - binary_crossentropy: 0.0777 - jaccard_coef_int: 0.0557 - binary_accuracy: 0.9846 - val_loss: 4.4275 - val_binary_crossentropy: 0.1052 - val_jaccard_coef_int: 0.0170 - val_binary_accuracy: 0.9806
Epoch 8/50
100/100 [==============================] - 11s 105ms/step - loss: 2.8917 - binary_crossentropy: 0.0615 - jaccard_coef_int: 0.0803 - binary_accuracy: 0.9894 - val_loss: 7.5899 - val_binary_crossentropy: 0.0829 - val_jaccard_coef_int: 1.8256e-14 - val_binary_accuracy: 0.9887
Epoch 9/50
100/100 [==============================] - 10s 105ms/step - loss: 2.7860 - binary_crossentropy: 0.0741 - jaccard_coef_int: 0.0868 - binary_accuracy: 0.9885 - val_loss: 4.1485 - val_binary_crossentropy: 0.0887 - val_jaccard_coef_int: 0.0203 - val_binary_accuracy: 0.9880
Epoch 10/50
100/100 [==============================] - 10s 104ms/step - loss: 2.6117 - binary_crossentropy: 0.0723 - jaccard_coef_int: 0.1018 - binary_ac

Epoch 36/50
100/100 [==============================] - 11s 105ms/step - loss: 4.6528 - binary_crossentropy: 0.0545 - jaccard_coef_int: 0.0048 - binary_accuracy: 0.9926 - val_loss: 5.1414 - val_binary_crossentropy: 0.1268 - val_jaccard_coef_int: 0.0041 - val_binary_accuracy: 0.9855
Epoch 37/50
100/100 [==============================] - 10s 105ms/step - loss: 4.6231 - binary_crossentropy: 0.0536 - jaccard_coef_int: 0.0059 - binary_accuracy: 0.9927 - val_loss: 4.6631 - val_binary_crossentropy: 0.0852 - val_jaccard_coef_int: 1.6731e-14 - val_binary_accuracy: 0.9873
Epoch 38/50
100/100 [==============================] - 11s 105ms/step - loss: 4.5830 - binary_crossentropy: 0.0519 - jaccard_coef_int: 0.0042 - binary_accuracy: 0.9928 - val_loss: 4.6524 - val_binary_crossentropy: 0.0815 - val_jaccard_coef_int: 1.7339e-14 - val_binary_accuracy: 0.9878
Epoch 39/50
100/100 [==============================] - 10s 105ms/step - loss: 4.5603 - binary_crossentropy: 0.0506 - jaccard_coef_int: 0.0051 - bi

...

100/100 [==============================] - 11s 107ms/step - loss: 1.9742 - binary_crossentropy: 0.2453 - jaccard_coef_int: 0.2153 - binary_accuracy: 0.9737 - val_loss: 5.2576 - val_binary_crossentropy: 0.1350 - val_jaccard_coef_int: 0.0062 - val_binary_accuracy: 0.9854
Epoch 30/50
100/100 [==============================] - 11s 106ms/step - loss: 1.9295 - binary_crossentropy: 0.2398 - jaccard_coef_int: 0.2232 - binary_accuracy: 0.9745 - val_loss: 2.6892 - val_binary_crossentropy: 0.2178 - val_jaccard_coef_int: 0.0987 - val_binary_accuracy: 0.9773
Epoch 31/50
100/100 [==============================] - 10s 105ms/step - loss: 1.8890 - binary_crossentropy: 0.2433 - jaccard_coef_int: 0.2309 - binary_accuracy: 0.9745 - val_loss: 3.7941 - val_binary_crossentropy: 0.1239 - val_jaccard_coef_int: 0.0279 - val_binary_accuracy: 0.9865
Epoch 32/50
100/100 [==============================] - 11s 106ms/step - loss: 1.8618 - binary_crossentropy: 0.2352 - jaccard_coef_int: 0.2339 - binary_accuracy: 0.975

Epoch 6/50
100/100 [==============================] - 9s 93ms/step - loss: 3.1982 - binary_crossentropy: 0.1834 - jaccard_coef_int: 0.0812 - binary_accuracy: 0.9609 - val_loss: 8.0842 - val_binary_crossentropy: 0.1473 - val_jaccard_coef_int: 1.0421e-14 - val_binary_accuracy: 0.9813
Epoch 7/50
100/100 [==============================] - 9s 94ms/step - loss: 3.0923 - binary_crossentropy: 0.2007 - jaccard_coef_int: 0.0871 - binary_accuracy: 0.9622 - val_loss: 3.5635 - val_binary_crossentropy: 0.1503 - val_jaccard_coef_int: 0.0429 - val_binary_accuracy: 0.9762
Epoch 8/50
100/100 [==============================] - 9s 93ms/step - loss: 3.0665 - binary_crossentropy: 0.2159 - jaccard_coef_int: 0.0889 - binary_accuracy: 0.9624 - val_loss: 3.3757 - val_binary_crossentropy: 0.2124 - val_jaccard_coef_int: 0.0626 - val_binary_accuracy: 0.9634
Epoch 9/50
100/100 [==============================] - 9s 94ms/step - loss: 3.0115 - binary_crossentropy: 0.2201 - jaccard_coef_int: 0.0910 - binary_accuracy: 0

Epoch 12/50
100/100 [==============================] - 10s 99ms/step - loss: 3.6584 - binary_crossentropy: 0.5965 - jaccard_coef_int: 0.1063 - binary_accuracy: 0.9135 - val_loss: 7.8147 - val_binary_crossentropy: 0.4608 - val_jaccard_coef_int: 5.6429e-04 - val_binary_accuracy: 0.9413
Epoch 13/50
100/100 [==============================] - 10s 99ms/step - loss: 3.5799 - binary_crossentropy: 0.5974 - jaccard_coef_int: 0.1095 - binary_accuracy: 0.9166 - val_loss: 5.1576 - val_binary_crossentropy: 0.6231 - val_jaccard_coef_int: 0.0177 - val_binary_accuracy: 0.9283
Epoch 14/50
100/100 [==============================] - 10s 98ms/step - loss: 3.5457 - binary_crossentropy: 0.6106 - jaccard_coef_int: 0.1118 - binary_accuracy: 0.9170 - val_loss: 5.1700 - val_binary_crossentropy: 0.8806 - val_jaccard_coef_int: 0.0377 - val_binary_accuracy: 0.8821
Epoch 15/50
100/100 [==============================] - 10s 100ms/step - loss: 3.4435 - binary_crossentropy: 0.6406 - jaccard_coef_int: 0.1220 - binary_ac

Epoch 18/50
100/100 [==============================] - 11s 107ms/step - loss: 3.0629 - binary_crossentropy: 0.5887 - jaccard_coef_int: 0.1439 - binary_accuracy: 0.9310 - val_loss: 4.3683 - val_binary_crossentropy: 0.6228 - val_jaccard_coef_int: 0.0388 - val_binary_accuracy: 0.9297
Epoch 19/50
100/100 [==============================] - 11s 108ms/step - loss: 3.0443 - binary_crossentropy: 0.5923 - jaccard_coef_int: 0.1458 - binary_accuracy: 0.9314 - val_loss: 3.9124 - val_binary_crossentropy: 0.6583 - val_jaccard_coef_int: 0.0680 - val_binary_accuracy: 0.9233
Epoch 20/50
100/100 [==============================] - 11s 107ms/step - loss: 2.9056 - binary_crossentropy: 0.5808 - jaccard_coef_int: 0.1599 - binary_accuracy: 0.9341 - val_loss: 4.4890 - val_binary_crossentropy: 0.6464 - val_jaccard_coef_int: 0.0338 - val_binary_accuracy: 0.9309
Epoch 21/50
100/100 [==============================] - 11s 108ms/step - loss: 2.9092 - binary_crossentropy: 0.5941 - jaccard_coef_int: 0.1603 - binary_acc

Epoch 24/50
100/100 [==============================] - 10s 95ms/step - loss: 2.4308 - binary_crossentropy: 0.0918 - jaccard_coef_int: 0.1679 - binary_accuracy: 0.9885 - val_loss: 5.4560 - val_binary_crossentropy: 0.0447 - val_jaccard_coef_int: 0.1049 - val_binary_accuracy: 0.9923
Epoch 25/50
100/100 [==============================] - 9s 94ms/step - loss: 2.1272 - binary_crossentropy: 0.0943 - jaccard_coef_int: 0.2044 - binary_accuracy: 0.9886 - val_loss: 5.2591 - val_binary_crossentropy: 0.0087 - val_jaccard_coef_int: 0.5804 - val_binary_accuracy: 0.9987
Epoch 26/50
100/100 [==============================] - 10s 95ms/step - loss: 2.3919 - binary_crossentropy: 0.0996 - jaccard_coef_int: 0.1672 - binary_accuracy: 0.9879 - val_loss: 5.6545 - val_binary_crossentropy: 0.0176 - val_jaccard_coef_int: 0.5661 - val_binary_accuracy: 0.9978
Epoch 27/50
100/100 [==============================] - 10s 95ms/step - loss: 3.3520 - binary_crossentropy: 0.1403 - jaccard_coef_int: 0.1264 - binary_accuracy

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/rgbtest/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                       repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw[:,:,:,0:3] / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 1
{1: 1.0}
composition with label != 0
Y_train 0.8582930756843801 0.2713395732689211
composition with label != 0
Y_val 0.8428571428571429 0.2126607142857143
composition with label != 0
Y_test 0.861271676300578 0.2920628612716763
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 3) (70, 112, 112, 3) (173, 112, 112, 3) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., steps_per_epoch=100, verbose=1, epochs=50, callbacks=[<keras.ca..., validation_data=(array([[[...)`


Epoch 1/50
100/100 [==============================] - 47s 471ms/step - loss: 6.0959 - binary_crossentropy: 0.6790 - jaccard_coef_int: 0.2187 - binary_accuracy: 0.7496 - val_loss: 14.1272 - val_binary_crossentropy: 4.5754 - val_jaccard_coef_int: 0.2135 - val_binary_accuracy: 0.2165
Epoch 2/50
100/100 [==============================] - 10s 97ms/step - loss: 4.5596 - binary_crossentropy: 0.8796 - jaccard_coef_int: 0.3790 - binary_accuracy: 0.7840 - val_loss: 12.9487 - val_binary_crossentropy: 4.0697 - val_jaccard_coef_int: 0.2215 - val_binary_accuracy: 0.2883
Epoch 3/50
100/100 [==============================] - 9s 95ms/step - loss: 4.4474 - binary_crossentropy: 0.9938 - jaccard_coef_int: 0.3864 - binary_accuracy: 0.7861 - val_loss: 6.8560 - val_binary_crossentropy: 1.6438 - val_jaccard_coef_int: 0.3106 - val_binary_accuracy: 0.6458
Epoch 4/50
100/100 [==============================] - 10s 96ms/step - loss: 4.2564 - binary_crossentropy: 1.0614 - jaccard_coef_int: 0.3956 - binary_accuracy:

Epoch 7/50
100/100 [==============================] - 10s 100ms/step - loss: 4.3850 - binary_crossentropy: 0.7407 - jaccard_coef_int: 0.1195 - binary_accuracy: 0.8636 - val_loss: 7.2573 - val_binary_crossentropy: 0.7569 - val_jaccard_coef_int: 0.0035 - val_binary_accuracy: 0.8769
Epoch 8/50
100/100 [==============================] - 10s 101ms/step - loss: 4.3368 - binary_crossentropy: 0.7593 - jaccard_coef_int: 0.1163 - binary_accuracy: 0.8679 - val_loss: 6.8135 - val_binary_crossentropy: 1.1755 - val_jaccard_coef_int: 0.0124 - val_binary_accuracy: 0.8444
Epoch 9/50
100/100 [==============================] - 10s 99ms/step - loss: 4.1345 - binary_crossentropy: 0.7761 - jaccard_coef_int: 0.1247 - binary_accuracy: 0.8750 - val_loss: 7.2417 - val_binary_crossentropy: 0.9638 - val_jaccard_coef_int: 0.0059 - val_binary_accuracy: 0.8607
Epoch 10/50
100/100 [==============================] - 10s 100ms/step - loss: 4.1299 - binary_crossentropy: 0.8048 - jaccard_coef_int: 0.1262 - binary_accurac

Epoch 13/50
100/100 [==============================] - 9s 95ms/step - loss: 3.8748 - binary_crossentropy: 0.1884 - jaccard_coef_int: 0.0444 - binary_accuracy: 0.9684 - val_loss: 5.2244 - val_binary_crossentropy: 0.1780 - val_jaccard_coef_int: 0.0071 - val_binary_accuracy: 0.9799
Epoch 14/50
100/100 [==============================] - 9s 95ms/step - loss: 3.4703 - binary_crossentropy: 0.2285 - jaccard_coef_int: 0.0557 - binary_accuracy: 0.9706 - val_loss: 3.4998 - val_binary_crossentropy: 0.3211 - val_jaccard_coef_int: 0.0590 - val_binary_accuracy: 0.9603
Epoch 15/50
100/100 [==============================] - 9s 95ms/step - loss: 3.2025 - binary_crossentropy: 0.2205 - jaccard_coef_int: 0.0642 - binary_accuracy: 0.9761 - val_loss: 3.5184 - val_binary_crossentropy: 0.4902 - val_jaccard_coef_int: 0.0637 - val_binary_accuracy: 0.9515
Epoch 16/50
100/100 [==============================] - 10s 95ms/step - loss: 3.1473 - binary_crossentropy: 0.2523 - jaccard_coef_int: 0.0708 - binary_accuracy: 

...

Epoch 2/50
100/100 [==============================] - 10s 104ms/step - loss: 2.6783 - binary_crossentropy: 0.1613 - jaccard_coef_int: 0.1690 - binary_accuracy: 0.9626 - val_loss: 5.5723 - val_binary_crossentropy: 0.0774 - val_jaccard_coef_int: 0.0029 - val_binary_accuracy: 0.9871
Epoch 3/50
100/100 [==============================] - 10s 103ms/step - loss: 2.4083 - binary_crossentropy: 0.1805 - jaccard_coef_int: 0.1853 - binary_accuracy: 0.9640 - val_loss: 4.5883 - val_binary_crossentropy: 0.0847 - val_jaccard_coef_int: 0.0133 - val_binary_accuracy: 0.9866
Epoch 4/50
100/100 [==============================] - 10s 102ms/step - loss: 2.3552 - binary_crossentropy: 0.2079 - jaccard_coef_int: 0.1850 - binary_accuracy: 0.9643 - val_loss: 3.0382 - val_binary_crossentropy: 0.1095 - val_jaccard_coef_int: 0.0681 - val_binary_accuracy: 0.9821
Epoch 5/50
100/100 [==============================] - 10s 103ms/step - loss: 2.2287 - binary_crossentropy: 0.2089 - jaccard_coef_int: 0.2017 - binary_accurac

Epoch 8/50
100/100 [==============================] - 10s 102ms/step - loss: 2.9896 - binary_crossentropy: 0.9239 - jaccard_coef_int: 0.6024 - binary_accuracy: 0.8461 - val_loss: 3.1710 - val_binary_crossentropy: 1.0344 - val_jaccard_coef_int: 0.6250 - val_binary_accuracy: 0.8324
Epoch 9/50
100/100 [==============================] - 10s 101ms/step - loss: 2.9768 - binary_crossentropy: 0.9702 - jaccard_coef_int: 0.6032 - binary_accuracy: 0.8466 - val_loss: 2.9290 - val_binary_crossentropy: 1.0391 - val_jaccard_coef_int: 0.6121 - val_binary_accuracy: 0.8481
Epoch 10/50
100/100 [==============================] - 10s 102ms/step - loss: 2.9272 - binary_crossentropy: 0.9990 - jaccard_coef_int: 0.6112 - binary_accuracy: 0.8487 - val_loss: 2.9160 - val_binary_crossentropy: 1.0484 - val_jaccard_coef_int: 0.6341 - val_binary_accuracy: 0.8468
Epoch 11/50
100/100 [==============================] - 10s 102ms/step - loss: 2.9177 - binary_crossentropy: 1.0204 - jaccard_coef_int: 0.6175 - binary_accur

Epoch 14/50
100/100 [==============================] - 10s 102ms/step - loss: 0.6744 - binary_crossentropy: 0.0437 - jaccard_coef_int: 0.6396 - binary_accuracy: 0.9932 - val_loss: 1.1394 - val_binary_crossentropy: 0.1105 - val_jaccard_coef_int: 0.4097 - val_binary_accuracy: 0.9855
Epoch 15/50
100/100 [==============================] - 10s 103ms/step - loss: 0.7195 - binary_crossentropy: 0.0441 - jaccard_coef_int: 0.6515 - binary_accuracy: 0.9932 - val_loss: 1.0997 - val_binary_crossentropy: 0.1107 - val_jaccard_coef_int: 0.4237 - val_binary_accuracy: 0.9859
Epoch 16/50
100/100 [==============================] - 10s 102ms/step - loss: 0.5957 - binary_crossentropy: 0.0416 - jaccard_coef_int: 0.6644 - binary_accuracy: 0.9937 - val_loss: 1.2159 - val_binary_crossentropy: 0.1188 - val_jaccard_coef_int: 0.3818 - val_binary_accuracy: 0.9847
Epoch 17/50
100/100 [==============================] - 10s 102ms/step - loss: 0.6278 - binary_crossentropy: 0.0477 - jaccard_coef_int: 0.6499 - binary_acc

Epoch 20/50
100/100 [==============================] - 10s 101ms/step - loss: 2.6241 - binary_crossentropy: 0.0979 - jaccard_coef_int: 0.1197 - binary_accuracy: 0.9845 - val_loss: 3.1814 - val_binary_crossentropy: 0.2362 - val_jaccard_coef_int: 0.0765 - val_binary_accuracy: 0.9633
Epoch 21/50
100/100 [==============================] - 11s 106ms/step - loss: 2.6041 - binary_crossentropy: 0.0859 - jaccard_coef_int: 0.1362 - binary_accuracy: 0.9863 - val_loss: 4.0011 - val_binary_crossentropy: 0.2421 - val_jaccard_coef_int: 0.0367 - val_binary_accuracy: 0.9570
Epoch 22/50
100/100 [==============================] - 10s 103ms/step - loss: 2.4484 - binary_crossentropy: 0.0887 - jaccard_coef_int: 0.1437 - binary_accuracy: 0.9867 - val_loss: 6.8765 - val_binary_crossentropy: 0.3671 - val_jaccard_coef_int: 0.0024 - val_binary_accuracy: 0.9481
Epoch 23/50
100/100 [==============================] - 10s 102ms/step - loss: 2.4275 - binary_crossentropy: 0.1003 - jaccard_coef_int: 0.1471 - binary_acc

Epoch 49/50
100/100 [==============================] - 10s 103ms/step - loss: 1.7671 - binary_crossentropy: 0.1005 - jaccard_coef_int: 0.2301 - binary_accuracy: 0.9885 - val_loss: 9.8602 - val_binary_crossentropy: 0.1305 - val_jaccard_coef_int: 1.3391e-14 - val_binary_accuracy: 0.9857
Epoch 50/50
100/100 [==============================] - 11s 107ms/step - loss: 1.6935 - binary_crossentropy: 0.0938 - jaccard_coef_int: 0.2431 - binary_accuracy: 0.9897 - val_loss: 5.1801 - val_binary_crossentropy: 0.1625 - val_jaccard_coef_int: 0.0068 - val_binary_accuracy: 0.9796
0.9912093569364162
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00   1100808
        1.0       0.04      0.02      0.03      6392

avg / total       0.99      0.99      0.99   1107200





In [13]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/rgbtest/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                       repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw[:,:,:,0:3] / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 3) (70, 112, 112, 3) (173, 112, 112, 3) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., epochs=50, steps_per_epoch=100, verbose=1, validation_data=(array([[[..., callbacks=[<keras.ca...)`


Epoch 1/50
100/100 [==============================] - 18s 179ms/step - loss: 3.0312 - binary_crossentropy: 0.0844 - jaccard_coef_int: 0.4702 - binary_accuracy: 0.9810 - val_loss: 1.2351 - val_binary_crossentropy: 0.0652 - val_jaccard_coef_int: 0.6384 - val_binary_accuracy: 0.9905
Epoch 2/50
100/100 [==============================] - 9s 93ms/step - loss: 1.1638 - binary_crossentropy: 0.0312 - jaccard_coef_int: 0.6883 - binary_accuracy: 0.9937 - val_loss: 0.8414 - val_binary_crossentropy: 0.0630 - val_jaccard_coef_int: 0.6432 - val_binary_accuracy: 0.9907
Epoch 3/50
100/100 [==============================] - 9s 88ms/step - loss: 0.8160 - binary_crossentropy: 0.0265 - jaccard_coef_int: 0.7255 - binary_accuracy: 0.9946 - val_loss: 1.0803 - val_binary_crossentropy: 0.0621 - val_jaccard_coef_int: 0.4827 - val_binary_accuracy: 0.9873
Epoch 4/50
100/100 [==============================] - 9s 90ms/step - loss: 0.5728 - binary_crossentropy: 0.0250 - jaccard_coef_int: 0.7403 - binary_accuracy: 0.9

100/100 [==============================] - 9s 89ms/step - loss: 0.3769 - binary_crossentropy: 0.0400 - jaccard_coef_int: 0.7479 - binary_accuracy: 0.9956 - val_loss: 0.6927 - val_binary_crossentropy: 0.1344 - val_jaccard_coef_int: 0.6082 - val_binary_accuracy: 0.9880
Epoch 31/50
100/100 [==============================] - 9s 88ms/step - loss: 0.3595 - binary_crossentropy: 0.0411 - jaccard_coef_int: 0.7608 - binary_accuracy: 0.9954 - val_loss: 0.5687 - val_binary_crossentropy: 0.1071 - val_jaccard_coef_int: 0.6557 - val_binary_accuracy: 0.9910
Epoch 32/50
100/100 [==============================] - 9s 88ms/step - loss: 0.3527 - binary_crossentropy: 0.0450 - jaccard_coef_int: 0.7634 - binary_accuracy: 0.9953 - val_loss: 0.5678 - val_binary_crossentropy: 0.1146 - val_jaccard_coef_int: 0.6565 - val_binary_accuracy: 0.9909
Epoch 33/50
100/100 [==============================] - 9s 88ms/step - loss: 0.3593 - binary_crossentropy: 0.0430 - jaccard_coef_int: 0.7595 - binary_accuracy: 0.9953 - val_

Epoch 7/50
100/100 [==============================] - 10s 103ms/step - loss: 4.6457 - binary_crossentropy: 1.2060 - jaccard_coef_int: 0.2935 - binary_accuracy: 0.7879 - val_loss: 5.0888 - val_binary_crossentropy: 1.1826 - val_jaccard_coef_int: 0.1518 - val_binary_accuracy: 0.8003
Epoch 8/50
100/100 [==============================] - 10s 103ms/step - loss: 4.6539 - binary_crossentropy: 1.2900 - jaccard_coef_int: 0.2981 - binary_accuracy: 0.7861 - val_loss: 4.7375 - val_binary_crossentropy: 1.3323 - val_jaccard_coef_int: 0.2510 - val_binary_accuracy: 0.7912
Epoch 9/50
100/100 [==============================] - 10s 103ms/step - loss: 4.6011 - binary_crossentropy: 1.3411 - jaccard_coef_int: 0.3029 - binary_accuracy: 0.7882 - val_loss: 4.6421 - val_binary_crossentropy: 1.3302 - val_jaccard_coef_int: 0.2683 - val_binary_accuracy: 0.7930
Epoch 10/50
100/100 [==============================] - 10s 101ms/step - loss: 4.5673 - binary_crossentropy: 1.3850 - jaccard_coef_int: 0.3017 - binary_accura

...

Epoch 11/50
100/100 [==============================] - 9s 89ms/step - loss: 2.9701 - binary_crossentropy: 0.5868 - jaccard_coef_int: 0.1911 - binary_accuracy: 0.9195 - val_loss: 3.6206 - val_binary_crossentropy: 0.6777 - val_jaccard_coef_int: 0.1144 - val_binary_accuracy: 0.9095
Epoch 12/50
100/100 [==============================] - 9s 91ms/step - loss: 2.8160 - binary_crossentropy: 0.5794 - jaccard_coef_int: 0.2077 - binary_accuracy: 0.9237 - val_loss: 3.4172 - val_binary_crossentropy: 0.8480 - val_jaccard_coef_int: 0.1920 - val_binary_accuracy: 0.8902
Epoch 13/50
100/100 [==============================] - 9s 90ms/step - loss: 2.7813 - binary_crossentropy: 0.5947 - jaccard_coef_int: 0.2147 - binary_accuracy: 0.9235 - val_loss: 4.5651 - val_binary_crossentropy: 1.5957 - val_jaccard_coef_int: 0.1998 - val_binary_accuracy: 0.8165
Epoch 14/50
100/100 [==============================] - 9s 90ms/step - loss: 2.7502 - binary_crossentropy: 0.5874 - jaccard_coef_int: 0.2150 - binary_accuracy: 0

100/100 [==============================] - 9s 90ms/step - loss: 1.6375 - binary_crossentropy: 0.5301 - jaccard_coef_int: 0.4277 - binary_accuracy: 0.9511 - val_loss: 3.2841 - val_binary_crossentropy: 0.9518 - val_jaccard_coef_int: 0.1516 - val_binary_accuracy: 0.9125
Epoch 41/50
100/100 [==============================] - 9s 90ms/step - loss: 1.6108 - binary_crossentropy: 0.5237 - jaccard_coef_int: 0.4332 - binary_accuracy: 0.9520 - val_loss: 3.3365 - val_binary_crossentropy: 0.9406 - val_jaccard_coef_int: 0.1414 - val_binary_accuracy: 0.9135
Epoch 42/50
100/100 [==============================] - 9s 90ms/step - loss: 1.5444 - binary_crossentropy: 0.5094 - jaccard_coef_int: 0.4502 - binary_accuracy: 0.9537 - val_loss: 3.1971 - val_binary_crossentropy: 0.9572 - val_jaccard_coef_int: 0.1605 - val_binary_accuracy: 0.9144
Epoch 43/50
100/100 [==============================] - 9s 90ms/step - loss: 1.4935 - binary_crossentropy: 0.5082 - jaccard_coef_int: 0.4705 - binary_accuracy: 0.9540 - val_

Epoch 17/50
100/100 [==============================] - 9s 91ms/step - loss: 3.6973 - binary_crossentropy: 0.0756 - jaccard_coef_int: 0.0889 - binary_accuracy: 0.9872 - val_loss: 31.1089 - val_binary_crossentropy: 0.0252 - val_jaccard_coef_int: 8.9879e-14 - val_binary_accuracy: 0.9958
Epoch 18/50
100/100 [==============================] - 9s 90ms/step - loss: 3.3800 - binary_crossentropy: 0.0721 - jaccard_coef_int: 0.1027 - binary_accuracy: 0.9886 - val_loss: 31.2613 - val_binary_crossentropy: 0.0204 - val_jaccard_coef_int: 6.1935e-14 - val_binary_accuracy: 0.9956
Epoch 19/50
100/100 [==============================] - 9s 91ms/step - loss: 3.5935 - binary_crossentropy: 0.0767 - jaccard_coef_int: 0.1108 - binary_accuracy: 0.9877 - val_loss: 30.0496 - val_binary_crossentropy: 0.0123 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9983
Epoch 20/50
100/100 [==============================] - 9s 92ms/step - loss: 3.6016 - binary_crossentropy: 0.0715 - jaccard_coef_int: 0.1048 - binary_

100/100 [==============================] - 9s 89ms/step - loss: 4.7354 - binary_crossentropy: 0.0644 - jaccard_coef_int: 0.0943 - binary_accuracy: 0.9906 - val_loss: 29.3830 - val_binary_crossentropy: 0.0052 - val_jaccard_coef_int: 0.7125 - val_binary_accuracy: 0.9997
Epoch 47/50
100/100 [==============================] - 9s 91ms/step - loss: 4.3568 - binary_crossentropy: 0.0629 - jaccard_coef_int: 0.1000 - binary_accuracy: 0.9910 - val_loss: 30.3530 - val_binary_crossentropy: 0.0079 - val_jaccard_coef_int: 0.6875 - val_binary_accuracy: 0.9996
Epoch 48/50
100/100 [==============================] - 9s 91ms/step - loss: 4.5631 - binary_crossentropy: 0.0744 - jaccard_coef_int: 0.0965 - binary_accuracy: 0.9893 - val_loss: 30.3552 - val_binary_crossentropy: 0.0089 - val_jaccard_coef_int: 0.6125 - val_binary_accuracy: 0.9990
Epoch 49/50
100/100 [==============================] - 9s 90ms/step - loss: 3.1512 - binary_crossentropy: 0.0788 - jaccard_coef_int: 0.1236 - binary_accuracy: 0.9881 - v

Epoch 23/50
100/100 [==============================] - 9s 92ms/step - loss: 2.4950 - binary_crossentropy: 0.0696 - jaccard_coef_int: 0.1237 - binary_accuracy: 0.9927 - val_loss: 11.3378 - val_binary_crossentropy: 0.0293 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9977
Epoch 24/50
100/100 [==============================] - 9s 90ms/step - loss: 2.3261 - binary_crossentropy: 0.0644 - jaccard_coef_int: 0.1313 - binary_accuracy: 0.9929 - val_loss: 10.7947 - val_binary_crossentropy: 0.0263 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9977
Epoch 25/50
100/100 [==============================] - 9s 91ms/step - loss: 2.1216 - binary_crossentropy: 0.0664 - jaccard_coef_int: 0.1483 - binary_accuracy: 0.9935 - val_loss: 3.2683 - val_binary_crossentropy: 0.0426 - val_jaccard_coef_int: 0.0404 - val_binary_accuracy: 0.9956
Epoch 26/50
100/100 [==============================] - 9s 91ms/step - loss: 2.6676 - binary_crossentropy: 0.0943 - jaccard_coef_int: 0.1017 - binary_accuracy:

In [27]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/rgbtest/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw[:,:,:,0:3] / 65535

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 3) (70, 112, 112, 3) (173, 112, 112, 3) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., epochs=50, steps_per_epoch=100, verbose=1, validation_data=(array([[[..., callbacks=[<keras.ca...)`


Epoch 1/50
100/100 [==============================] - 47s 473ms/step - loss: 4.2104 - binary_crossentropy: 0.2923 - jaccard_coef_int: 0.2025 - binary_accuracy: 0.8932 - val_loss: 4.5383 - val_binary_crossentropy: 0.4214 - val_jaccard_coef_int: 0.0667 - val_binary_accuracy: 0.8941
Epoch 2/50
100/100 [==============================] - 9s 93ms/step - loss: 2.6787 - binary_crossentropy: 0.2954 - jaccard_coef_int: 0.3938 - binary_accuracy: 0.9109 - val_loss: 5.5529 - val_binary_crossentropy: 0.4379 - val_jaccard_coef_int: 0.0101 - val_binary_accuracy: 0.9061
Epoch 3/50
100/100 [==============================] - 9s 93ms/step - loss: 2.4443 - binary_crossentropy: 0.3452 - jaccard_coef_int: 0.4092 - binary_accuracy: 0.9150 - val_loss: 7.4161 - val_binary_crossentropy: 0.8240 - val_jaccard_coef_int: 3.5956e-04 - val_binary_accuracy: 0.8895
Epoch 4/50
100/100 [==============================] - 9s 93ms/step - loss: 2.0303 - binary_crossentropy: 0.3193 - jaccard_coef_int: 0.4630 - binary_accuracy:

100/100 [==============================] - 9s 94ms/step - loss: 1.0808 - binary_crossentropy: 0.3657 - jaccard_coef_int: 0.6581 - binary_accuracy: 0.9594 - val_loss: 1.9340 - val_binary_crossentropy: 0.8695 - val_jaccard_coef_int: 0.5172 - val_binary_accuracy: 0.9218
Epoch 31/50
100/100 [==============================] - 10s 95ms/step - loss: 1.0370 - binary_crossentropy: 0.3498 - jaccard_coef_int: 0.6652 - binary_accuracy: 0.9613 - val_loss: 1.2079 - val_binary_crossentropy: 0.3153 - val_jaccard_coef_int: 0.5874 - val_binary_accuracy: 0.9584
Epoch 32/50
100/100 [==============================] - 9s 94ms/step - loss: 0.9946 - binary_crossentropy: 0.3349 - jaccard_coef_int: 0.6785 - binary_accuracy: 0.9627 - val_loss: 1.3863 - val_binary_crossentropy: 0.3764 - val_jaccard_coef_int: 0.5384 - val_binary_accuracy: 0.9527
Epoch 33/50
100/100 [==============================] - 9s 94ms/step - loss: 0.9748 - binary_crossentropy: 0.3303 - jaccard_coef_int: 0.6826 - binary_accuracy: 0.9635 - val

Epoch 7/50
100/100 [==============================] - 10s 98ms/step - loss: 2.5977 - binary_crossentropy: 0.2554 - jaccard_coef_int: 0.2086 - binary_accuracy: 0.9416 - val_loss: 2.9237 - val_binary_crossentropy: 0.3100 - val_jaccard_coef_int: 0.1584 - val_binary_accuracy: 0.9359
Epoch 8/50
100/100 [==============================] - 10s 97ms/step - loss: 2.5217 - binary_crossentropy: 0.2661 - jaccard_coef_int: 0.2183 - binary_accuracy: 0.9424 - val_loss: 2.9101 - val_binary_crossentropy: 0.3120 - val_jaccard_coef_int: 0.1518 - val_binary_accuracy: 0.9390
Epoch 9/50
100/100 [==============================] - 10s 98ms/step - loss: 2.4143 - binary_crossentropy: 0.2491 - jaccard_coef_int: 0.2453 - binary_accuracy: 0.9427 - val_loss: 2.3404 - val_binary_crossentropy: 0.2372 - val_jaccard_coef_int: 0.2867 - val_binary_accuracy: 0.9372
Epoch 10/50
100/100 [==============================] - 10s 99ms/step - loss: 2.1004 - binary_crossentropy: 0.2116 - jaccard_coef_int: 0.3159 - binary_accuracy: 

...

100/100 [==============================] - 10s 101ms/step - loss: 1.2316 - binary_crossentropy: 0.3712 - jaccard_coef_int: 0.5778 - binary_accuracy: 0.9578 - val_loss: 1.5068 - val_binary_crossentropy: 0.4492 - val_jaccard_coef_int: 0.4961 - val_binary_accuracy: 0.9499
Epoch 40/50
100/100 [==============================] - 10s 102ms/step - loss: 1.2152 - binary_crossentropy: 0.3697 - jaccard_coef_int: 0.5818 - binary_accuracy: 0.9583 - val_loss: 1.5885 - val_binary_crossentropy: 0.4377 - val_jaccard_coef_int: 0.4444 - val_binary_accuracy: 0.9516
Epoch 41/50
100/100 [==============================] - 10s 103ms/step - loss: 1.1891 - binary_crossentropy: 0.3609 - jaccard_coef_int: 0.5872 - binary_accuracy: 0.9594 - val_loss: 1.4280 - val_binary_crossentropy: 0.4454 - val_jaccard_coef_int: 0.5201 - val_binary_accuracy: 0.9520
Epoch 42/50
100/100 [==============================] - 10s 103ms/step - loss: 1.2086 - binary_crossentropy: 0.3754 - jaccard_coef_int: 0.5866 - binary_accuracy: 0.958

Epoch 16/50
100/100 [==============================] - 11s 108ms/step - loss: 1.6733 - binary_crossentropy: 0.3052 - jaccard_coef_int: 0.3653 - binary_accuracy: 0.9595 - val_loss: 1.8748 - val_binary_crossentropy: 0.2913 - val_jaccard_coef_int: 0.2816 - val_binary_accuracy: 0.9631
Epoch 17/50
100/100 [==============================] - 11s 107ms/step - loss: 1.6463 - binary_crossentropy: 0.3106 - jaccard_coef_int: 0.3738 - binary_accuracy: 0.9597 - val_loss: 1.8160 - val_binary_crossentropy: 0.2967 - val_jaccard_coef_int: 0.2981 - val_binary_accuracy: 0.9634
Epoch 18/50
100/100 [==============================] - 11s 108ms/step - loss: 1.6308 - binary_crossentropy: 0.3128 - jaccard_coef_int: 0.3775 - binary_accuracy: 0.9600 - val_loss: 1.7711 - val_binary_crossentropy: 0.2808 - val_jaccard_coef_int: 0.2984 - val_binary_accuracy: 0.9656
Epoch 19/50
100/100 [==============================] - 11s 108ms/step - loss: 1.5927 - binary_crossentropy: 0.3074 - jaccard_coef_int: 0.3836 - binary_acc

100/100 [==============================] - 11s 108ms/step - loss: 1.2412 - binary_crossentropy: 0.3194 - jaccard_coef_int: 0.4741 - binary_accuracy: 0.9694 - val_loss: 1.5912 - val_binary_crossentropy: 0.3009 - val_jaccard_coef_int: 0.3253 - val_binary_accuracy: 0.9709
Epoch 46/50
100/100 [==============================] - 11s 108ms/step - loss: 1.2110 - binary_crossentropy: 0.3172 - jaccard_coef_int: 0.4855 - binary_accuracy: 0.9698 - val_loss: 1.5847 - val_binary_crossentropy: 0.3164 - val_jaccard_coef_int: 0.3326 - val_binary_accuracy: 0.9701
Epoch 47/50
100/100 [==============================] - 11s 108ms/step - loss: 1.2203 - binary_crossentropy: 0.3246 - jaccard_coef_int: 0.4846 - binary_accuracy: 0.9694 - val_loss: 1.5891 - val_binary_crossentropy: 0.3326 - val_jaccard_coef_int: 0.3356 - val_binary_accuracy: 0.9692
Epoch 48/50
100/100 [==============================] - 11s 108ms/step - loss: 1.2208 - binary_crossentropy: 0.3283 - jaccard_coef_int: 0.4843 - binary_accuracy: 0.969

Epoch 22/50
100/100 [==============================] - 10s 104ms/step - loss: 1.1013 - binary_crossentropy: 0.2774 - jaccard_coef_int: 0.6809 - binary_accuracy: 0.9566 - val_loss: 1.2452 - val_binary_crossentropy: 0.3298 - val_jaccard_coef_int: 0.6657 - val_binary_accuracy: 0.9490
Epoch 23/50
100/100 [==============================] - 10s 104ms/step - loss: 1.0657 - binary_crossentropy: 0.2718 - jaccard_coef_int: 0.6880 - binary_accuracy: 0.9581 - val_loss: 1.2745 - val_binary_crossentropy: 0.3796 - val_jaccard_coef_int: 0.6606 - val_binary_accuracy: 0.9477
Epoch 24/50
100/100 [==============================] - 10s 104ms/step - loss: 1.0878 - binary_crossentropy: 0.2856 - jaccard_coef_int: 0.6828 - binary_accuracy: 0.9571 - val_loss: 1.1913 - val_binary_crossentropy: 0.3406 - val_jaccard_coef_int: 0.6773 - val_binary_accuracy: 0.9513
Epoch 25/50
100/100 [==============================] - 10s 104ms/step - loss: 1.0562 - binary_crossentropy: 0.2799 - jaccard_coef_int: 0.6910 - binary_acc

             precision    recall  f1-score   support

        0.0       0.98      0.97      0.98    980995
        1.0       0.81      0.84      0.83    126205

avg / total       0.96      0.96      0.96   1107200



